In [1]:
import os
import lasio
import pandas as pd
import plotly.graph_objs as go
from ipywidgets import interact, widgets
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed

In [2]:
def find_las_files(folder):
    las_files = []
    for dirpath, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename.endswith('.las'):
                las_files.append(os.path.join(dirpath, filename))
    return las_files




def extract_log_names(las_file):
    try:
        las = lasio.read(las_file, engine='normal')
        return las_file, [curve.mnemonic for curve in las.curves]
    except Exception as e:
        print(f"Failed to read {las_file}: {str(e)}")
        return las_file, []






In [3]:
folder = r'E:\Data\Wells'  # Replace with your folder path
las_files = find_las_files(folder)

n_jobs = -1  # Use all available CPU cores
data = []

results = Parallel(n_jobs=n_jobs)(delayed(extract_log_names)(las_file) for las_file in las_files)

for las_file, log_names in results:
    for log_name in log_names:
        data.append({'LAS_File': las_file, 'Log_Name': log_name})

df = pd.DataFrame(data)


In [4]:
log_categories = {
    'Density': ['RHOB', 'ZDEN', 'DEN', 'DPHI', 'DPHZ', 'DPOR', 'RHOZ', 'RHGF', 'RHG', 'RHGA', 'RHGD', 'RHGM', 'RHGX'],
    'Gamma Ray': ['GR', 'GAMMA', 'CGR', 'SGR', 'HCGR', 'HSGR', 'GRR', 'GRS', 'GRD', 'GRN', 'GRC', 'GR_EDTC', 'GR_STGC'],
    'Resistivity': ['RES', 'RT', 'RLLD', 'RLLS', 'MSFL', 'ILD', 'ILM', 'LLD', 'LLS', 'LL8', 'LL7', 'ATRT', 'AT90', 'RWA', 'R2', 'R1', 'RESD', 'RESM', 'RESS', 'RLL3'],
    'Neutron Porosity': ['NPHI', 'TNPH', 'NPOR', 'CNC', 'CNPOR', 'SNP', 'NPHS', 'NPOR_DSN', 'NPRL', 'NPLS'],
    'Sonic': ['DT', 'DTC', 'DTSM', 'DTS', 'DTSF', 'DTS_SLOW', 'DT1', 'DT2', 'DT4P', 'DT4S', 'DTRP', 'DTTP', 'DTR', 'DTRS', 'DT24', 'DTCO', 'DTMD', 'DTLF', 'DTLN', 'DTLO', 'DTSW'],
    'Caliper': ['CAL', 'CALI', 'LCAL', 'MCAL', 'CALS', 'HCAL', 'CALD', 'CALN', 'CALR', 'CALX', 'C1', 'C2', 'C3'],
    'SP': ['SP', 'SPSB', 'SSPK', 'SPC', 'SPD'],
    'PEF': ['PEF', 'PE', 'PEFZ', 'PEDN'],
    'Acoustic': ['AC', 'AC1', 'AC2', 'ACD', 'ACS', 'ACTT', 'ACXP', 'AE20', 'AE30', 'AE60', 'AHTCO60'],
    'Bit Size': ['BS', 'BSZ'],
    'Formation Micro-Imager': ['FMI', 'FMI_HILT', 'FMIH', 'FMSF', 'FMSC', 'FMSX', 'FMS'],
    'Temperature': ['BHT', 'MTEM', 'HTNP'],
    'Pressure': ['MPR', 'BHP', 'HTNP_PRES'],
    'Deep Reading': ['DRHO', 'DRH', 'DRHB', 'ZDNC'],
    'Shallow Reading': ['SRAT', 'SRAT_SRM', 'SRXP', 'SRLC', 'SFLU', 'SFBC', 'SFL', 'SFLR'],
    'Medium Reading': ['MRAT', 'MRIL', 'MRXP', 'MRES', 'MRR', 'RMED'],
    'Photoelectric Absorption': ['UZ', 'UAPI', 'UPOR'],
    'Thermal Decay Time': ['TDT', 'TDTF'],
    'Natural Gamma Ray Spectroscopy': ['THOR', 'POTA', 'URAN'],
    'Dielectric Constant': ['DIEC', 'DEPO'],
    'Azimuthal Resistivity': ['AZIM', 'AZI'],
    'p_wave_tools': ['DT', 'DTC', 'DTSM', 'DTS', 'DTSF', 'DTS_SLOW', 'DT1', 'DT2', 'DT4P', 'DTRP', 'DTTP', 'DTR', 'DTCO', 'DTMD', 'DTLF', 'DTLN', 'DTLO', 'DTSW'],
    's_wave_tools': ['DT4S', 'DTRS', 'DT24']
}


In [5]:
category_checkboxes = [widgets.Checkbox(value=False, description=category) for category in log_categories.keys()]
checkbox_widgets = widgets.VBox(category_checkboxes)


In [6]:
button = widgets.Button(description="Find Wells")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        selected_categories = [checkbox.description for checkbox in category_checkboxes if checkbox.value]
        if not selected_categories:
            print("Please select at least one log category.")
            return
        
        grouped = df.groupby('LAS_File')['Log_Name'].apply(set).reset_index()
        wells_with_selected_logs = []
        for i, row in grouped.iterrows():
            if all(any(log in row['Log_Name'] for log in log_categories[category]) for category in selected_categories):
                wells_with_selected_logs.append(row['LAS_File'])

        print("Wells with selected logs:")
        for well in wells_with_selected_logs:
            print(well)

button.on_click(on_button_click)




In [7]:
widgets.VBox([checkbox_widgets, button, output])


In [ ]:
df

In [ ]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

# Create a new Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active
ws.title = "Wells"

# Write the DataFrame to the worksheet
for r_idx, row in enumerate(df.to_numpy(), start=2):
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Write the header row
for c_idx, col_name in enumerate(df.columns, start=1):
    ws.cell(row=1, column=c_idx, value=col_name)

# Create a table with filters
tab = Table(displayName="Table1", ref=ws.dimensions)
tab.tableStyleInfo = TableStyleInfo(
    name="TableStyleMedium9", showFirstColumn=False,
    showLastColumn=False, showRowStripes=True, showColumnStripes=True)
ws.add_table(tab)

# Save the workbook
wb.save("Wells.xlsx")


In [ ]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

log_categories = {
    # ... (the same dictionary as you provided earlier)
}

# Create a function to categorize the logs
def categorize_log(log_name):
    for category, names in log_categories.items():
        if log_name in names:
            return category
    return 'Other'

# Add a new column to the DataFrame with the log categories
df['Category'] = df['Log_Name'].apply(categorize_log)

# Create a new Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active
ws.title = "Wells"

# Write the DataFrame to the worksheet
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=1):
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Create a table with filters
tab = Table(displayName="Table1", ref=ws.dimensions)
tab.tableStyleInfo = TableStyleInfo(
    name="TableStyleMedium9", showFirstColumn=False,
    showLastColumn=False, showRowStripes=True, showColumnStripes=True)
ws.add_table(tab)

# Save the workbook
wb.save("Wells.xlsx")


In [ ]:
df


In [ ]:
import shutil

def copy_files_to_directory(file_paths, dest_folder):
    # Loop through the list of file paths and copy each one to the destination folder
    for file_path in file_paths:
        # Get the name of the file from its path
        file_name = file_path.split("\\")[-1]
        
        # Create the destination file path
        dest_path = f"{dest_folder}\\{file_name}"
        
        # Copy the file to the destination folder
        shutil.copy(file_path, dest_path)

# List of file paths you want to copy
file_paths = [
    "E:\\Data\\Wells\\43-13a-5\\WirelineData\\Depth\\43-13a-5 30155_d.las",
    "E:\\Data\\Wells\\43-13b-6Z\\WirelineData\\Depth\\43-13b-6Z 500799_d.las",
    "E:\\Data\\Wells\\43-19a-4\\WirelineData\\Depth\\43-19a-4 24257_d.las",
    "E:\\Data\\Wells\\43-28a-3\\WirelineData\\Depth\\43-28a-3 19629_d.las",
    "E:\\Data\\Wells\\44-12a-3\\WirelineData\\Depth\\44-12a-3 30202_d.las",
    "E:\\Data\\Wells\\44-12a-6\\WirelineData\\Depth\\44-12a-6 502894_d.las",
    "E:\\Data\\Wells\\44-23b-11\\WirelineData\\Depth\\44-23b-11 29226_d.las",
    "E:\\Data\\Wells\\47-10-8\\WirelineData\\Depth\\47-10-8 29129_d.las",
    "E:\\Data\\Wells\\47-4a-7\\WirelineData\\Depth\\47-4a-7 11601_d.las",
    "E:\\Data\\Wells\\48-12a-7Z\\WirelineData\\Depth\\48-12a-7Z 19743_d.las",
    "E:\\Data\\Wells\\48-12a-8\\WirelineData\\Depth\\48-12a-8 22025_d.las",
    "E:\\Data\\Wells\\48-18c-14\\WirelineData\\Depth\\48-18c-14 25546_d.las",
    "E:\\Data\\Wells\\48-20a-7\\WirelineData\\Depth\\48-20a-7 23077_d.las",
    "E:\\Data\\Wells\\48-23b-6\\WirelineData\\Depth\\48-23b-6 27890_d.las",
    "E:\\Data\\Wells\\48-2b-3\\WirelineData\\Depth\\48-2b-3 25646_d.las",
    "E:\\Data\\Wells\\48-6-42\\WirelineData\\Depth\\48-6-42 29646_d.las",
    "E:\\Data\\Wells\\49-20a-7\\WirelineData\\Depth\\49-20a-7 29431_d.las",
    "E:\\Data\\Wells\\49-29a-10\\WirelineData\\Depth\\49-29a-10 20699_d.las",
    "E:\\Data\\Wells\\49-2a-5Z\\WirelineData\\Depth\\49-2a-5Z 20748_d.las",
    "E:\\Data\\Wells\\49-9b-6Z\\WirelineData\\Depth\\49-9b-6Z 13406_d.las",
    "E:\\Data\\Wells\\53-15a-1\\WirelineData\\Depth\\53-15a-1 28638_d.las",
    "E:\\Data\\Wells\\53-3a-7\\WirelineData\\Depth\\53-3a-7 28967_d.las",
    "E:\\Data\\Wells\\53-5b-6\\WirelineData\\Depth\\53-5b-6 26958_d.las",
    "E:\\Data\\Wells\\54-1b-6\\WirelineData\\Depth\\54-1b-6 29554_d.las"
]

# Destination folder
dest_folder = "E:\\WELLSCHECK"

# Execute the function to copy the files
copy_files_to_directory(file_paths, dest_folder)

print("Files have been copied.")
